# Import

In [1]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
from scripts.evalModel import evalLogisticModels, getPredDist

# Load data

## Cell lines

In [3]:
trainRNA = pd.read_csv('../../data/processed/RNA_train_cancergenes.csv', index_col=0)
trainCellLines = list(trainRNA.index)

testRNA = pd.read_csv('../../data/processed/RNA_test_cancergenes.csv', index_col=0)
testCellLines = list(testRNA.index)

newRNA = pd.read_csv('../../data/processed/RNA_newcancer_cancergenes.csv', index_col=0)
newCellLines = list(newRNA.index)

## CDR

In [4]:
cdr = pd.read_csv('../../data/processed/drugCellLinePairsData.csv', index_col='DepMap_ID')
trainCDR = cdr.loc[trainCellLines, :].reset_index()
testCDR = cdr.loc[testCellLines, :].reset_index()
newCDR = cdr.loc[newCellLines, :].reset_index()

In [5]:
testTemp = testCDR.loc[:, ['DepMap_ID', 'cancer_type', 'name', 'effective']].rename(columns={'DepMap_ID':'cell_line',
                                                                                             'name': 'drug',
                                                                                             'effective': 'true'})

newTemp = newCDR.loc[:, ['DepMap_ID', 'cancer_type', 'name', 'effective']].rename(columns={'DepMap_ID':'cell_line',
                                                                                          'name': 'drug',
                                                                                          'effective': 'true'})

## Drugs

In [6]:
drugs = pd.read_csv('../../data/processed/drug_fingerprints.csv', index_col=0)

In [7]:
trainDrugs = drugs.loc[list(trainCDR.name.values), :].to_numpy()
testDrugs = drugs.loc[list(testCDR.name.values), :].to_numpy()
newDrugs = drugs.loc[list(newCDR.name.values), :].to_numpy()

trainRNA = trainRNA.loc[list(trainCDR.DepMap_ID.values), :].to_numpy()
testRNA = testRNA.loc[list(testCDR.DepMap_ID.values), :].to_numpy()
newRNA = newRNA.loc[list(newCDR.DepMap_ID.values), :].to_numpy()

## Format Pairs

In [8]:
trainData = [trainDrugs, trainRNA]
trainEff = trainCDR.effective.to_numpy()
testData = [testDrugs, testRNA]
newData = [newDrugs, newRNA]

In [9]:
del cdr, drugs, trainDrugs, testDrugs, newDrugs, trainRNA, testRNA, newRNA

# Model

In [10]:
drugPath = '/fs/scratch/PCON0041/PatrickLawrence/cancer-drug-response/fewShot/drugs/siameseV1/models/'
drugModel = 'DrugFewShot_Layers1_Hidden64_DO0-1_AFrelu_LR0-001_DR0-99_DS1000/'
drugPath = os.path.join(drugPath, drugModel)

In [11]:
rnaPath = '/fs/scratch/PCON0041/PatrickLawrence/cancer-drug-response/fewShot/cellLines/siameseV1/models/'
rnaModel = 'CellLineFewShot_Layers2_Hidden64_DO0-1_AFrelu_LR0-001_DR0-99_DS1000/'
rnaPath = os.path.join(rnaPath, rnaModel)

In [12]:
evalLog = evalLogisticModels(trainData.copy(), trainEff, testData.copy(), newData.copy(), alt='xgb',
                            fusionPath=None, drugPath=drugPath, rnaPath=rnaPath)

In [20]:
evalLog.model.best_params_

{'learning_rate': 0.01, 'max_depth': 7, 'objective': 'binary:logistic'}

## Cell line performance

In [13]:
testDF, newDF, testWrong, newWrong, testCounts, newCounts = evalLog.evaluate(testDF=testTemp.copy(), 
                                                                             newDF=newTemp.copy())

Average Cell Line precision @ k on test set
	Precision@1: 0.9608
	Precision@2: 0.9412
	Precision@3: 0.8824
	Precision@4: 0.848
	Precision@5: 0.8392
	Precision@10: 0.8436

Average Cell Line precision @ k on newcancer set
	Precision@1: 0.9538
	Precision@2: 0.9308
	Precision@3: 0.8923
	Precision@4: 0.8769
	Precision@5: 0.8492
	Precision@10: 0.8275


Test set:

	# of cell lines without effective drug among top-3 recs: 0
	# of unique drugs among top-3 predictions: 14

New cancer set
No true effective drugs identified in top 3 for ACH-000250 (top drug: dolastatin-10)
No true effective drugs identified in top 3 for ACH-000268 (top drug: BGT226)

	# of cell lines without effective drug among top-3 recs: 2
	# of unique drugs among top-3 predictions: 15


## Pred distribution

In [14]:
testDist = getPredDist(testDF)

Avg varaince of predictions for each drug: 9.999999747378752e-05


In [15]:
newDist = getPredDist(newDF)

Avg varaince of predictions for each drug: 9.999999747378752e-05


## Cancer performance

In [16]:
testCancer, newCancer = evalLog.getCancerPerformance(testDF, newDF)

In [17]:
print(round(testCancer.mean(), 4))
testCancer

p1    0.9658
p2    0.9505
p3    0.8831
p4    0.8506
p5    0.8443
dtype: float64


,p1,p2,p3,p4,p5
Liver Cancer,1.000000,1.000000,1.000000,1.000000,1.000000
Colon/Colorectal Cancer,1.000000,1.000000,1.000000,1.000000,0.950000
Head and Neck Cancer,1.000000,1.000000,1.000000,0.916667,0.866667
Bladder Cancer,1.000000,1.000000,0.888889,0.916667,0.933333
Endometrial/Uterine Cancer,1.000000,1.000000,0.888889,0.833333,0.866667
Ovarian Cancer,1.000000,1.000000,0.833333,0.812500,0.850000
Breast Cancer,1.000000,1.000000,0.777778,0.750000,0.733333
Skin Cancer,1.000000,0.900000,0.866667,0.800000,0.720000
Pancreatic Cancer,1.000000,0.875000,0.833333,0.812500,0.750000
Brain Cancer,1.000000,0.875000,0.833333,0.750000,0.800000


In [18]:
print(round(newCancer.mean(), 4))
newCancer

p1    0.9769
p2    0.9440
p3    0.9057
p4    0.8737
p5    0.8494
dtype: float64


,p1,p2,p3,p4,p5
Prostate Cancer,1.000000,1.000000,1.000000,1.000000,0.900000
Sarcoma,1.000000,1.000000,1.000000,1.000000,0.866667
Gallbladder Cancer,1.000000,1.000000,1.000000,0.750000,0.800000
Gastric Cancer,1.000000,1.000000,0.928571,0.928571,0.900000
Bone Cancer,1.000000,1.000000,0.925926,0.888889,0.888889
Rhabdoid,1.000000,1.000000,0.916667,0.875000,0.900000
Thyroid Cancer,1.000000,0.937500,0.916667,0.875000,0.900000
Bile Duct Cancer,1.000000,0.900000,0.933333,0.900000,0.800000
Neuroblastoma,1.000000,0.833333,0.666667,0.750000,0.800000
Kidney Cancer,0.769231,0.769231,0.769231,0.769231,0.738462
